## 합성 테스트 데이터셋 생성

In [1]:
import langchain
import ragas

print(f"LangChain Version: {langchain.__version__}")
print(f"Ragas Version: {ragas.__version__}")

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LangChain Version: 0.2.17
Ragas Version: 0.1.19


In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("./data.pdf")

# 문서 로딩
docs = loader.load()

# 문서의 페이지수
len(docs)

101

In [8]:
print(len(docs))

101


In [4]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [64]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document
import time

# Docx2txtLoader로 변경
loader = Docx2txtLoader("../dataset2.docx")
docs = loader.load()

# 작품명을 기준으로 문서 분리 함수
def split_by_artwork_title(docs):
    split_docs = []
    total_docs = len(docs)

    for idx, doc in enumerate(docs):
        content = doc.page_content  # 수정: Document 객체의 내용을 가져옴
        # 작품명을 기준으로 분리
        artworks = content.split("\n\n작품명:")
        for art in artworks:
            lines = art.strip().split("\n")
            title = "알 수 없음"
            if lines and lines[0].startswith("작품명:"):
                title = lines[0].replace("작품명:", "").strip()
            split_docs.append(Document(page_content=art.strip(), metadata={"작품명": title}))

        # 진행률 출력
        progress = ((idx + 1) / total_docs) * 100
        print(f"🔄 진행률: {progress:.2f}% 완료")
        time.sleep(0.1)  # 출력이 너무 빠르지 않도록 잠시 대기

    return split_docs

# 작품명을 기준으로 분리
docs = split_by_artwork_title(docs)
print(f"📄 분리된 문서 수: {len(docs)}")


🔄 진행률: 50.00% 완료
🔄 진행률: 100.00% 완료
📄 분리된 문서 수: 2


In [5]:
print(type(docs[50]))


<class 'langchain_core.documents.base.Document'>


In [6]:
docs[0].metadata

{'source': './data.pdf',
 'file_path': './data.pdf',
 'page': 0,
 'total_pages': 101,
 'Producer': 'iLovePDF',
 'ModDate': 'D:20250112103612Z',
 'filename': './data.pdf'}

In [7]:

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [8]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

In [9]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

In [10]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 1}

In [11]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=20,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

embedding nodes:   8%|▊         | 17/220 [00:01<00:20,  9.87it/s]Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage limit exceeded"}')trace=96bceb3b-457b-4d47-878d-3f03d05ea60e,id=96bceb3b-457b-4d47-878d-3f03d05ea60e; trace=07e47844-1160-40fc-94c1-26ae3a5c3654,id=07e47844-1160-40fc-94c1-26ae3a5c3654; trace=6b33cf65-d473-40d7-ada8-9a8763d929a8,id=6b33cf65-d473-40d7-ada8-9a8763d929a8; trace=ca78c5d4-23fb-4ad6-87f7-0b3fdfb2d5af,id=ca78c5d4-23fb-4ad6-87f7-0b3fdfb2d5af; trace=8683d4de-32ff-45b0-85af-c005eef6a253,id=8683d4de-32ff-45b0-85af-c005eef6a253; trace=d01dfd1e-58a2-42a4-b780-66104ca28b06,id=d01dfd1e-58a2-42a4-b780-66104ca28b06; trace=9c8c05bc-9677-4507-86a1-ccb3a91d4f6a,id=9c8c05bc-9677-4507-86a1-ccb3a91d4f6a
Failed to multipart ingest runs: langsm

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-BX3Q3wXp4SIUltz1g5vEja6b on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage limit exceeded"}')trace=254c2ec6-1472-4dfe-984e-3dd60acea633,id=254c2ec6-1472-4dfe-984e-3dd60acea633; trace=59db534a-e822-47a8-9493-71e4c31f86b5,id=59db534a-e822-47a8-9493-71e4c31f86b5; trace=53f22ccc-a26a-4b20-815b-6d16ad5c372a,id=53f22ccc-a26a-4b20-815b-6d16ad5c372a; trace=bc0255c3-dd5a-4d55-bceb-40602a136fca,id=bc0255c3-dd5a-4d55-bceb-40602a136fca
Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Monthly unique traces usage limit exceeded"}')trace=83480e10-cd59-4fc9-8d9d-ab97dbd01e21,id=8

In [12]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

NameError: name 'testset' is not defined

In [18]:
# DataFrame의 상위 5개 행 출력
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What specific movements or styles emerged in v...,[• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연도...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
1,What significant art exhibition took place in ...,[• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 / ...,The significant art exhibition that took place...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
2,What is the significance of the song 'Graceful...,[• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villag...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
3,What is the significance of Korean traditional...,[• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• 재...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
4,What themes link 'Spring Dream' to modern lit?,"[산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리감...",The answer to given question is not present in...,reasoning,"[{'source': './data.pdf', 'file_path': './data...",True


In [20]:
# DataFrame을 CSV 파일로 저장
test_df.to_csv("./ragas_qa_0112.csv", index=False)

## RAGAS를 활용한 평가

In [21]:
!pip install -qU faiss-cpu ragas

In [137]:
import pandas as pd

df = pd.read_csv("ragas_translate_0112.csv")
df

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,"[""• 작품 번호: 172\n• 제작 연도: 1974\n• 크기: 36.5×20×2...",simple,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,'산천'이라는 제목의 작품이 갖는 의미는 무엇인가요?,'산천'이라는 제목의 작품이 갖는 의미는 문맥에서 드러나지 않습니다.
1,"['수상했다. 광복 후 대한민국미술전람회의 초대 작가 및 심사 위원을 역임하고,\n...",simple,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,춘향전의 맥락에서 조용승이 갖는 의미는 무엇인가요?,조용승은 1937년 출간된 '춘향전'을 각색한 '춘향이야기'의 작가라는 점에서 춘향...
2,['작가라는 점에는 이견이 없다. 그의 작품세계는 1970년대 중반 이후부터 많은\...,simple,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,한국 근대 미술 운동은 20세기 문화 지형에 어떤 중요한 공헌을 했나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,['• 작품명: 해초 따는 여인 / 海草 따는 女人 / Woman Taking Se...,simple,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,이순자 작가의 작품 '중첩 3번'의 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,"['수상했다. 광복 후 대한민국미술전람회의 초대 작가 및 심사 위원을 역임하고,\n...",reasoning,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,심형구 작가와 '포즈'라는 작품 사이에 문체적인 연관성이 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
5,['• 크기: 29×23×23\n• 재료: 청동에 채색(좌대: 대리석)\n• 카테고...,reasoning,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,45년부터 70년까지 김찬식의 주요 문학적 역할은 무엇이었나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
6,['수 있는 두터운 마티에르가 유지되고 있다. 실제 풍경을 그렸다기보다 자연을\n관...,multi_context,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,"20세기 한국화, 특히 60년대와 70년대에는 어떤 주요한 변화와 작품이 있었나요?","20세기 한국 회화를 형성한 다양한 주요 변화와 작품, 특히 김환기 같은 작가의 영..."
7,"['• 재료: 종이에 먹, 색\n• 카테고리: 회화 I\n• 작품 설명: 설봉(雪峰...",conditional,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,20세기 초 한국 미술과 오리엔탈리즘에서 '춘몽'은 어떤 역할을 했나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
8,['• 재료: 대리석\n• 카테고리: 조각ㆍ설치\n• 작품 설명: 김정숙(1917-...,conditional,"[{'source': 'data/data.pdf', 'file_path': 'dat...",True,이순자의 '1978년 2월의 숲'은 인포멀과 미니멀리즘과 관련해 어떤 의미가 있을까요?,인포멜과 미니멀리즘과 관련하여 이승자의 '1978년 2월의 숲'이 갖는 의미는 두 ...


In [123]:
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,['• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,"1950년대와 1960년대에 시각 예술에 어떤 특정한 사조나 스타일이 등장했으며, ...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,"[""• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 /...",simple,"[{'source': './data.pdf', 'file_path': './data...",True,1976년에 어떤 중요한 미술 전시회가 열렸나요?,1976년에 열린 이 중요한 미술 전시회의 제목은 '형상을 제안하다'(그림 I)입니다.
2,['• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villa...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,'우아한 세상'이라는 곡이 발매와 테마의 맥락에서 어떤 의미가 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,['• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• ...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,한국 전통 공연이 문화 표현에서 갖는 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,"['산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리...",reasoning,"[{'source': './data.pdf', 'file_path': './data...",True,'봄의 꿈'과 현대 조명을 연결하는 테마는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [140]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 9
})

In [141]:
import ast


# contexts 컬럼의 문자열을 리스트로 변환
def convert_to_list(example):
    contexts = ast.literal_eval(example["contexts"])
    return {"contexts": contexts}


test_dataset = test_dataset.map(convert_to_list)
print(test_dataset)

Map: 100%|██████████| 9/9 [00:00<00:00, 2322.86 examples/s]

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 9
})


In [142]:
test_dataset[1]["contexts"]

['수상했다. 광복 후 대한민국미술전람회의 초대 작가 및 심사 위원을 역임하고,\n서울대학교와 홍익대학교에서 교수로 재직했다.일제강점기에는 향토적인 소재를\n채색공필화법을 사용하여 그렸고 광복 이후에는 일본 색을 멀리하고 문인화를\n제작했다.<승무도>는 제16회 조선미술전람회(1937)에서 입선한 작품으로,\n장우성의 초기 작품 경향을 알 수 있는 작품이다. 민속무용 중 하나인 ‘승무(僧舞)’는\n인물화에 능한 화가들이 즐겨 택했던 주제중의 하나로, 작가는 흰 고깔을 쓴 승려가\n양손에 북채를 쥔 채 춤추는 모습을 그렸다. 다음 동작으로 넘어가려는 듯한 자세를\n취하고 있어 긴장감을 조성하며, 파계승(破戒僧)의 폭발할 듯한 번뇌까지도\n응축되어 있다. 엄격한 데생과 세필, 화려한 색채 등에서 김은호의 영향을 알 수\n있다.\n• 작품명: 춘향전 / 春香傳 / Chun-Hyang Story\n• 작가: 조용승 / CHO Yongseung\n• 작품 번호: 188\n• 제작 연도: 1937\n• 크기: 141.7×157\n• 재료: 비단에 먹, 색\n• 카테고리: 회화 I\n• 작품 설명: 예당(藝堂) 조용승(曺龍承, 1912-1946)은 김은호에게 사사했고, 김은호의\n문하생들이 결성한 단체인 ‘후소회(後素會)’의 회원으로 활동했다. 광복 후,\n1946년에는 한국 문화의 정체성을 찾고자 하는 미술단체인 ‘단구미술원(檀丘美術\n院)’을 조직하여 활동했다. 그는 인물화에 특히 뛰어났는데, 김은호의 영향을 받아\n일본식 채색화풍을 따랐다.고전 소설인 ‘춘향전’을 주제로 그린 <춘향전>은 한복을\n곱게 차려 입은 주인공을 부드러운 세필로 머리카락, 발 등을 섬세하게 묘사했다.\n일본화풍을 띠며, 촛대, 소반 등의 전통적인 한국적 기물을 함께 구성하여 작품\n소재와 배경의 조화를 꾀하고 있다.']

In [103]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [104]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:39<00:00,  5.62s/it]


In [106]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda


In [107]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks. 
Answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with "The document does not contain this information." 
Provide detailed and comprehensive answers, always include the artwork number, and ensure all answers are written in Korean. 
All answers should be formatted using beautiful Markdown syntax to make the response visually appealing and easy to read. Use headings, bullet points, and bold or italic text where appropriate to enrich the response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''


# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [108]:
import re
class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"### \n\n{extracted_text}\n\n"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"### \n\n{llm_output.strip()}\n\n"


In [109]:
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

# 2. FAISS 데이터베이스 로드
persist_directory = "../faiss_artworks_0113"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [110]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.5        # 다양성과 관련성 간 균형
    }
)


In [111]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap
# 단계 8: 체인(Chain) 생성
chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser()                    # 응답을 문자열로 변환
)

In [121]:
response = chain.invoke({"question": "낙양성동도이화작품번호는?"})
print(response)

### 

**낙양성동도이화 작품 번호:**  **1364**




In [143]:
batch_dataset = [question for question in test_dataset["question"]]
batch_dataset[:3]

["'산천'이라는 제목의 작품이 갖는 의미는 무엇인가요?",
 '춘향전의 맥락에서 조용승이 갖는 의미는 무엇인가요?',
 '한국 근대 미술 운동은 20세기 문화 지형에 어떤 중요한 공헌을 했나요?']

In [17]:
import os

# LangChain Tracing 비활성화
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_ENDPOINT"] = ""


In [144]:
answer = chain.batch(batch_dataset)
answer[:3]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


["### \n\n# '산천' 작품의 의미\n\n문서에 명시적으로 '산천'이라는 제목의 작품에 대한 정보는 제공되어 있지 않습니다. 따라서, 주어진 정보만으로는 해당 작품의 구체적인 의미나 내용을 설명하기 어렵습니다. 만약 특정 작품에 대한 질문이라면, 해당 작품의 세부 정보가 포함된 문서가 필요합니다. 현재 제공된 정보 내에서는 다음과 같이 일반적인 해석을 제시할 수 있습니다:\n\n- **자연과의 조화**: '산천'이라는 제목은 자연의 아름다움, 특히 산과 물의 조화를 표현하고 있을 가능성이 큽니다. 산수화의 전통적인 주제 중 하나가 자연의 아름다움과 조화로움이기 때문입니다.\n  \n- **정신적 평화**: 산과 물은 동양 미술에서 정신적 평화와 내면의 고요함을 상징하기도 합니다. 따라서 이 작품은 관람자에게 평온함과 정신적 안정감을 전달하려는 의도가 있을 수 있습니다.\n\n더 정확한 답변을 위해서는 '산천'이라는 제목의 작품에 대한 구체적인 정보가 필요합니다. 만약 특정 작품 번호나 추가 정보가 있다면 알려주시면 더욱 상세한 분석을 제공할 수 있습니다.\n\n**결론**:  \n*The document does not contain specific information about an artwork titled '산천'. Therefore, precise interpretation regarding its meaning cannot be provided without additional details.*\n\n",
 "### \n\n## 조용승의 의미: 춘향전 맥락에서\n\n조용승은 **조선시대 전통적인 회화 기법을 현대적으로 재해석**하며, 특히 **'춘향전'**을 주제로 한 작품에서 그 의미를 더욱 부각합니다. \n\n* **전통의 계승과 혁신:** 조용승은 고전 소설 '춘향전'을 바탕으로 한복을 입은 주인공을 섬세하게 묘사하며 전통적인 한국 미의식을 보여줍니다. 동시에 일본화풍을 접목시켜 새로운 해석을 시도하며 전통과 현대 사이의 균형을 추구했습

In [145]:
# 'answer' 컬럼 덮어쓰기 또는 추가
if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

In [146]:
test_dataset

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth', 'answer'],
    num_rows: 9
})

In [147]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating: 100%|██████████| 36/36 [00:56<00:00,  1.57s/it]


{'context_precision': 0.2778, 'faithfulness': 0.4018, 'answer_relevancy': 0.4951, 'context_recall': 0.7778}

In [148]:
result_df = result.to_pandas()
result_df.head()

,user_input,retrieved_contexts,response,reference,context_precision,faithfulness,answer_relevancy,context_recall
0,'산천'이라는 제목의 작품이 갖는 의미는 무엇인가요?,[• 작품 번호: 172\n• 제작 연도: 1974\n• 크기: 36.5×20×20...,### \n\n# '산천' 작품의 의미\n\n문서에 명시적으로 '산천'이라는 제목의...,'산천'이라는 제목의 작품이 갖는 의미는 문맥에서 드러나지 않습니다.,0.0,0.666667,0.000000,0.0
1,춘향전의 맥락에서 조용승이 갖는 의미는 무엇인가요?,"[수상했다. 광복 후 대한민국미술전람회의 초대 작가 및 심사 위원을 역임하고,\n서...",### \n\n## 조용승의 의미: 춘향전 맥락에서\n\n조용승은 **조선시대 전통...,조용승은 1937년 출간된 '춘향전'을 각색한 '춘향이야기'의 작가라는 점에서 춘향...,1.0,0.500000,0.833598,0.5
2,한국 근대 미술 운동은 20세기 문화 지형에 어떤 중요한 공헌을 했나요?,[작가라는 점에는 이견이 없다. 그의 작품세계는 1970년대 중반 이후부터 많은\n...,### \n\n## 한국 근대 미술 운동의 20세기 문화 지형에 대한 공헌\n\n한...,주어진 질문에 대한 답변이 컨텍스트에 없습니다.,0.0,0.000000,0.887446,1.0
3,이순자 작가의 작품 '중첩 3번'의 의미는 무엇인가요?,[• 작품명: 해초 따는 여인 / 海草 따는 女人 / Woman Taking Sea...,### \n\n**작품 정보:**\n- **제목:** 중첩 작품3 \n- **작가...,주어진 질문에 대한 답변이 컨텍스트에 없습니다.,0.0,0.470588,0.916761,1.0
4,심형구 작가와 '포즈'라는 작품 사이에 문체적인 연관성이 있나요?,"[수상했다. 광복 후 대한민국미술전람회의 초대 작가 및 심사 위원을 역임하고,\n서...","### \n\n**문체적 연관성 분석**\n\n심형구 작가와 그의 작품 **""<포즈...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.,0.0,0.764706,0.000000,1.0


In [135]:
result_df.to_csv("ragas_evaluation_0113.csv", index=False)

In [149]:
result_df.loc[:, "context_precision":"context_recall"]

,context_precision,faithfulness,answer_relevancy,context_recall
0,0.0,0.666667,0.000000,0.0
1,1.0,0.500000,0.833598,0.5
2,0.0,0.000000,0.887446,1.0
3,0.0,0.470588,0.916761,1.0
4,0.0,0.764706,0.000000,1.0
5,0.0,0.666667,0.898201,1.0
6,1.0,0.047619,0.919782,0.5
7,0.0,0.000000,0.000000,1.0
8,0.5,0.500000,0.000000,1.0


## DeepL을 사용해 데이터셋 번역

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
import pandas as pd

df = pd.read_csv("./ragas_qa_0113.csv")
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What specific movements or styles emerged in v...,['• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
1,What significant art exhibition took place in ...,"[""• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 /...",The significant art exhibition that took place...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
2,What is the significance of the song 'Graceful...,['• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villa...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
3,What is the significance of Korean traditional...,['• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• ...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True
4,What themes link 'Spring Dream' to modern lit?,"['산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리...",The answer to given question is not present in...,reasoning,"[{'source': './data.pdf', 'file_path': './data...",True


In [38]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [39]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

번역 진행 중: 100%|██████████| 9/9 [00:07<00:00,  1.27it/s]


In [40]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,question_translated,ground_truth_translated
0,What specific movements or styles emerged in v...,['• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,"1950년대와 1960년대에 시각 예술에 어떤 특정한 사조나 스타일이 등장했으며, ...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,What significant art exhibition took place in ...,"[""• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 /...",The significant art exhibition that took place...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,1976년에 어떤 중요한 미술 전시회가 열렸나요?,1976년에 열린 이 중요한 미술 전시회의 제목은 '형상을 제안하다'(그림 I)입니다.
2,What is the significance of the song 'Graceful...,['• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villa...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,'우아한 세상'이라는 곡이 발매와 테마의 맥락에서 어떤 의미가 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,What is the significance of Korean traditional...,['• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• ...,The answer to given question is not present in...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,한국 전통 공연이 문화 표현에서 갖는 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,What themes link 'Spring Dream' to modern lit?,"['산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리...",The answer to given question is not present in...,reasoning,"[{'source': './data.pdf', 'file_path': './data...",True,'봄의 꿈'과 현대 조명을 연결하는 테마는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [41]:
# question, ground_truth 열을 확인합니다.
df.loc[:, ["question", "ground_truth"]].head()

,question,ground_truth
0,What specific movements or styles emerged in v...,The answer to given question is not present in...
1,What significant art exhibition took place in ...,The significant art exhibition that took place...
2,What is the significance of the song 'Graceful...,The answer to given question is not present in...
3,What is the significance of Korean traditional...,The answer to given question is not present in...
4,What themes link 'Spring Dream' to modern lit?,The answer to given question is not present in...


In [42]:
# question_translated, ground_truth_translated 열을 확인합니다.
df.loc[:, ["question_translated", "ground_truth_translated"]].head()

,question_translated,ground_truth_translated
0,"1950년대와 1960년대에 시각 예술에 어떤 특정한 사조나 스타일이 등장했으며, ...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,1976년에 어떤 중요한 미술 전시회가 열렸나요?,1976년에 열린 이 중요한 미술 전시회의 제목은 '형상을 제안하다'(그림 I)입니다.
2,'우아한 세상'이라는 곡이 발매와 테마의 맥락에서 어떤 의미가 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,한국 전통 공연이 문화 표현에서 갖는 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,'봄의 꿈'과 현대 조명을 연결하는 테마는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [8]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,['• 작가: 김규진 / KIM Kyujin\n• 작품 번호: 279\n• 제작 연...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,"1950년대와 1960년대에 시각 예술에 어떤 특정한 사조나 스타일이 등장했으며, ...",주어진 질문에 대한 답변이 컨텍스트에 없습니다.
1,"[""• 작품명: 취상I / 取象I / Figuration I\n• 작가: 이종상 /...",simple,"[{'source': './data.pdf', 'file_path': './data...",True,1976년에 어떤 중요한 미술 전시회가 열렸나요?,1976년에 열린 이 중요한 미술 전시회의 제목은 '형상을 제안하다'(그림 I)입니다.
2,['• 작품명: 향가일취 / 鄕家逸趣 / A Scene of Native Villa...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,'우아한 세상'이라는 곡이 발매와 테마의 맥락에서 어떤 의미가 있나요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
3,['• 작품 번호: 186\n• 제작 연도: 1937\n• 크기: 71×51\n• ...,simple,"[{'source': './data.pdf', 'file_path': './data...",True,한국 전통 공연이 문화 표현에서 갖는 의미는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.
4,"['산수화 기법인 미점준(米點皴)을 사용했으며, 원근법을 사용하여 대상 간에\n거리...",reasoning,"[{'source': './data.pdf', 'file_path': './data...",True,'봄의 꿈'과 현대 조명을 연결하는 테마는 무엇인가요?,주어진 질문에 대한 답변이 컨텍스트에 없습니다.


In [43]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("./ragas_art_translate_0113.csv", index=False)

## HuggingFace에 Dataset 업로드

In [11]:
from datasets import Dataset

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 확인
print(dataset)

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 9
})


In [12]:
from datasets import Dataset
import os

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 이름 설정 (원하는 이름으로 변경하세요)
dataset_name = "charagas-test-dataseteeee/"

# 데이터셋 업로드
dataset.push_to_hub(
    dataset_name,
    private=True,  # private=False로 설정하면 공개 데이터셋이 됩니다.
    split="korean_v1",  # 데이터셋 split 이름 입력
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/chaeeee/ragas-test-dataset-0113/commit/b385cf5414f11d3234616cc9fa9b13b98be0b00e', commit_message='Upload dataset', commit_description='', oid='b385cf5414f11d3234616cc9fa9b13b98be0b00e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/chaeeee/ragas-test-dataset-0113', endpoint='https://huggingface.co', repo_type='dataset', repo_id='chaeeee/ragas-test-dataset-0113'), pr_revision=None, pr_num=None)

## 대용량 pdf 200페이지씩 분할

In [91]:
from pypdf import PdfReader, PdfWriter

def split_pdf(file_path, pages_per_split=200):
    # PDF 파일 열기
    reader = PdfReader(file_path)
    total_pages = len(reader.pages)

    # 몇 개의 파일로 나눌지 계산
    for i in range(0, total_pages, pages_per_split):
        writer = PdfWriter()
        split_start = i
        split_end = min(i + pages_per_split, total_pages)

        # 지정된 페이지 범위를 새로운 PDF에 추가
        for page_num in range(split_start, split_end):
            writer.add_page(reader.pages[page_num])

        # 결과 파일 저장
        output_filename = f"{file_path[:-4]}_part_{i//pages_per_split + 1}.pdf"
        with open(output_filename, "wb") as output_file:
            writer.write(output_file)
        
        print(f"{output_filename} 저장 완료! ({split_start + 1} ~ {split_end} 페이지)")

# PDF 파일 경로와 분할 페이지 수 입력
split_pdf("example.pdf", 200)


Note: you may need to restart the kernel to use updated packages.
